In [1]:
print("hello")

hello


In [3]:
import scipy
print("Scipy version:", scipy.__version__)


Scipy version: 1.15.2


In [17]:
import pyiast
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def binary_mixture(y_N2: float, y_CO2: float):

    df_N2 = pd.read_csv("N2_UTSA80.csv")
    df_CO2 = pd.read_csv("CO2_UTSA80.csv")

    CO2_isotherm = pyiast.InterpolatorIsotherm(df_CO2, loading_key="CO2 loading [mol/kg]", pressure_key="Pressure [bar]")
    N2_isotherm = pyiast.InterpolatorIsotherm(df_N2, loading_key="N2 loading [mol/kg]", pressure_key="Pressure [bar]")
    
    # Pressure range for this simulation
    pressure = [0.2, 1.01325]
    
    # Introduction of a binary mixture
    y = [y_N2, y_CO2]
    
    # Store results
    CO2_ads = []
    N2_ads = []
    total_loading = []

    for p in pressure:
        try: 
            # Partial pressure
            p_CO2 = p*y_CO2
            p_N2 = p*y_N2
            results = pyiast.iast([p_CO2, p_N2], [CO2_isotherm, N2_isotherm])
            CO2_ads.append(results[0])
            N2_ads.append(results[1])
            total_loading.append(sum(results))
            print (f"Pressure {p} bar: N2 = {results[1]: .4f}, CO2 = {results[0]: .4f}, Total = {sum(results):.4f}")

        except Exception as e: 
            print(f"Error at pressure {p} bar : {e}")
            N2_ads.append(np.nan)
            CO2_ads.append(np.nan)
            total_loading.append(np.nan)

    
    #Pressure = [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4]
    #CH4_IRMOF_data_not_mixed = [0.02266292256712, 0.08756719507668, 0.1564975455208, 0.22497333877876, 0.28796197743236, 0.357963784101, 0.42640710898852, 0.4826423265906]
    #CO2_IRMOF_data_not_mixed = [0.10091169532752, 0.39653620818372, 0.71810294902148, 1.0437607045347, 1.3820811245195, 1.7378045910601, 2.164016889826, 2.4447384206502]
    
    # Calculate total uptake
    #total_adsorption = [co2 + ch4 for co2, ch4 in zip(CO2_ads, CH4_ads)]

    # Plotting graph

    #plt.figure(figsize=(8,6))
    #plt.plot(pressure, CO2_ads, marker= ".", linewidth = 1, label = "CO2 uptake in a mixture", color = "darkblue")
    #plt.plot(pressure, N2_ads, marker = "x", markersize = 5, linewidth = 1, label = " CH4 uptake in a mixture", color = "darkgreen")
    #plt.plot(Pressure, CO2_IRMOF_data_not_mixed, linestyle = "--", label = "CO2 uptake if no mixture", color = "darkblue")
    #plt.plot(Pressure, CH4_IRMOF_data_not_mixed, linestyle = "--", label = "CH4 uptake if no mixture", color = "darkgreen")
    #plt.plot(pressure, total_adsorption, marker=" ", linestyle="-.", linewidth=1, label="Total uptake in mixture", color="purple")
    #plt.xlabel("Pressure [bar]")
    #plt.ylabel("Uptake [mol/kg]")
    #plt.title(f"Binary mixture adsorption isoterms {y_CH4}/{y_CO2} (CH4/CO2)")
    #plt.legend()
    #plt.tight_layout()
    #plt.show()

In [18]:
binary_mixture(0.85, 0.15)

Pressure 0.2 bar: N2 =  0.0571, CO2 =  0.0502, Total = 0.1073
Pressure 1.01325 bar: N2 =  0.2793, CO2 =  0.2485, Total = 0.5278


In [27]:
import pyiast
import pandas as pd

# Load isotherm data
df_N2 = pd.read_csv("N2_UTSA80.csv")
df_CO2 = pd.read_csv("CO2_UTSA80.csv")

# Create pyIAST isotherm objects
N2_isotherm = pyiast.InterpolatorIsotherm(df_N2, loading_key="N2 loading [mol/kg]", pressure_key="Pressure [bar]")
CO2_isotherm = pyiast.InterpolatorIsotherm(df_CO2, loading_key="CO2 loading [mol/kg]", pressure_key="Pressure [bar]")

# Mixture composition
y_N2 = 0.85
y_CO2 = 0.15

# Pressures of interest
pressure = [0.2, 1.01325]

print("Pressure (bar) | CO2 loading (mol/kg) | N2 loading (mol/kg) | Selectivity (CO2/CH4)")
print("-" * 70)
for p in pressure:
    # Partial pressures for each component
    p_CO2 = p * y_CO2
    p_N2 = p * y_N2
    # Get loadings from pyIAST
    results = pyiast.iast([p_CO2, p_N2], [CO2_isotherm, N2_isotherm])
    q_CO2 = results[0]
    q_N2 = results[1]
    # Calculate selectivity: S = (q_CO2 / y_CO2) / (q_CH4 / y_CH4)
    selectivity = (q_CO2 / y_CO2) / (q_N2 / y_N2)
    print(f"{p:>13} | {q_CO2:>18.4f} | {q_N2:>18.4f} | {selectivity:>22.2f}")

Pressure (bar) | CO2 loading (mol/kg) | N2 loading (mol/kg) | Selectivity (CO2/CH4)
----------------------------------------------------------------------
          0.2 |             0.0502 |             0.0571 |                   4.98
      1.01325 |             0.2485 |             0.2793 |                   5.04
